In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
data = pd.read_csv("./data/seattle-weather.csv")


data.head()

,date,precipitation,temp_max,temp_min,wind,weather
0,2012-01-01,0.0,12.8,5.0,4.7,drizzle
1,2012-01-02,10.9,10.6,2.8,4.5,rain
2,2012-01-03,0.8,11.7,7.2,2.3,rain
3,2012-01-04,20.3,12.2,5.6,4.7,rain
4,2012-01-05,1.3,8.9,2.8,6.1,rain


### Checking for missing values

In [3]:
data.isna().sum()

date             0
precipitation    0
temp_max         0
temp_min         0
wind             0
weather          0
dtype: int64

In [4]:
#No missing values in the dataset

### Preprocessing (date)

In [5]:
data["date"] = pd.to_datetime(data["date"])

In [6]:
data['year'] = data['date'].dt.year
data['month'] = data['date'].dt.month
data['day'] = data['date'].dt.day
data['day_of_week'] = data['date'].dt.dayofweek  # Monday=0, Sunday=6


data.drop("date", inplace=True, axis=1)


#### handling cylinical features (months)

In [7]:
data['month_sin'] = np.sin(2 * np.pi * data['month'] / 12)
data['month_cos'] = np.cos(2 * np.pi * data['month'] / 12)

data.drop("month", inplace=True, axis=1)


### now using label encoder to preprocess the output (weather)

In [8]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()

data["weather"] = label_encoder.fit_transform(data["weather"])


data.head()

,precipitation,temp_max,temp_min,wind,weather,year,day,day_of_week,month_sin,month_cos
0,0.0,12.8,5.0,4.7,0,2012,1,6,0.5,0.866025
1,10.9,10.6,2.8,4.5,2,2012,2,0,0.5,0.866025
2,0.8,11.7,7.2,2.3,2,2012,3,1,0.5,0.866025
3,20.3,12.2,5.6,4.7,2,2012,4,2,0.5,0.866025
4,1.3,8.9,2.8,6.1,2,2012,5,3,0.5,0.866025


#### shifiting "weather" column to the last position for easy visuals"

In [9]:
column_to_shift = 'weather'
columns = data.columns.tolist()
columns.remove(column_to_shift)
columns.append(column_to_shift)
data = data[columns]

In [10]:
data.head()

,precipitation,temp_max,temp_min,wind,year,day,day_of_week,month_sin,month_cos,weather
0,0.0,12.8,5.0,4.7,2012,1,6,0.5,0.866025,0
1,10.9,10.6,2.8,4.5,2012,2,0,0.5,0.866025,2
2,0.8,11.7,7.2,2.3,2012,3,1,0.5,0.866025,2
3,20.3,12.2,5.6,4.7,2012,4,2,0.5,0.866025,2
4,1.3,8.9,2.8,6.1,2012,5,3,0.5,0.866025,2


### Finally, scale some of the features

In [11]:
from sklearn.preprocessing import StandardScaler

features = ['precipitation', 'temp_max', 'temp_min', 'wind']

scaler = StandardScaler()

data[features] = scaler.fit_transform(data[features])

data.head()

,precipitation,temp_max,temp_min,wind,year,day,day_of_week,month_sin,month_cos,weather
0,-0.453650,-0.495299,-0.644212,1.014980,2012,1,6,0.5,0.866025,0
1,1.178598,-0.794731,-1.082347,0.875833,2012,2,0,0.5,0.866025,2
2,-0.333852,-0.645015,-0.206077,-0.654780,2012,3,1,0.5,0.866025,2
3,2.586224,-0.576962,-0.524720,1.014980,2012,4,2,0.5,0.866025,2
4,-0.258978,-1.026111,-1.082347,1.989006,2012,5,3,0.5,0.866025,2


### SPLITTING THE DATSET

In [12]:
X = data.drop("weather", axis=1)

y = data["weather"]


In [13]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)

In [14]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(1241, 9)
(1241,)
(220, 9)
(220,)


In [15]:
from keras.utils import to_categorical
y_train_categorical = to_categorical(y_train, num_classes=5)

# Reshape 'x_train' for LSTM input
x_train_reshaped = x_train.values.reshape((x_train.shape[0], x_train.shape[1], 1))

/Users/apple/Documents/campus security system/.venv/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


### BUild the BiLSTM model

In [45]:
from tensorflow import keras

from tensorflow.keras.layers import Bidirectional, LSTM, Dense, Embedding

In [52]:

model = keras.models.Sequential()

model.add(Bidirectional(LSTM(50, activation='relu'), input_shape=(x_train.shape[1], 1)))
model.add(Dense(5, activation='softmax'))  # Adjust the number of units based on your problem

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [53]:
model.fit(x_train_reshaped, y_train_categorical, epochs=18, batch_size=32, validation_split=0.2)

Epoch 1/18
31/31 [==============================] - 1s 13ms/step - loss: 2.9401 - accuracy: 0.4496 - val_loss: 1.1947 - val_accuracy: 0.6305
Epoch 2/18
31/31 [==============================] - 0s 5ms/step - loss: 1.0990 - accuracy: 0.5081 - val_loss: 1.1281 - val_accuracy: 0.4819
Epoch 3/18
31/31 [==============================] - 0s 5ms/step - loss: 1.2057 - accuracy: 0.5554 - val_loss: 1.0957 - val_accuracy: 0.4819
Epoch 4/18
31/31 [==============================] - 0s 5ms/step - loss: 1.0025 - accuracy: 0.6391 - val_loss: 0.9946 - val_accuracy: 0.5984
Epoch 5/18
31/31 [==============================] - 0s 4ms/step - loss: 0.9476 - accuracy: 0.6714 - val_loss: 0.9485 - val_accuracy: 0.6827
Epoch 6/18
31/31 [==============================] - 0s 4ms/step - loss: 0.9679 - accuracy: 0.6300 - val_loss: 0.9471 - val_accuracy: 0.6586
Epoch 7/18
31/31 [==============================] - 0s 4ms/step - loss: 0.9322 - accuracy: 0.6411 - val_loss: 0.9028 - val_accuracy: 0.6627
Epoch 8/18
31/31 [=

In [54]:
y_test_categorical = to_categorical(y_test, num_classes=5)

x_test_reshaped = x_test.values.reshape((x_test.shape[0], x_test.shape[1], 1))


loss, accuracy = model.evaluate(x_test_reshaped, y_test_categorical)

print(f'Test Loss: {loss * 100:.4f}, Test Accuracy: {accuracy * 100:.4f}')

7/7 [==============================] - 0s 2ms/step - loss: 0.7440 - accuracy: 0.7727
Test Loss: 74.4021, Test Accuracy: 77.2727


### Building the GAUSIAN NAIVE BAYES model

In [55]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

In [56]:
gnb_model = GaussianNB()

gnb_model.fit(x_train, y_train)

GaussianNB()

In [57]:
y_pred = gnb_model.predict(x_test)

accuracy = accuracy_score(y_test, y_pred)

print(f'Accuracy of GAUASIAN NAIVE BAYES: {accuracy}')

Accuracy of GAUASIAN NAIVE BAYES: 0.8590909090909091


### Using GAUSAIN PROCESS REGRESSION FOR HYPERPARAMETER TUNING 

The bayessian search cv uses the gaussian process regression for hyper parameter tuning of machine learning models

In [228]:
!pip install scikit-optimize

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.3/100.3 kB 4.3 kB/s eta 0:00:00a 0:00:02


In [30]:
!pip install scikeras


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip


The Sequential model from Keras does not have a 'score' method, which is necessary for the BayesSearchCV(Gaussian Process Regression)  to work. To do this we would need to create a wrapper around our model to allow scoring.

In [62]:

from scikeras.wrappers import KerasClassifier
from keras.models import Sequential
from keras.layers import Bidirectional, LSTM, Dense
from skopt import BayesSearchCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer


def create_model(units, dropout):
    model = Sequential()
    model.add(Bidirectional(LSTM(units, activation='relu'), input_shape=(9, 1)))
    model.add(Dense(5, activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model


# Wrap the Keras model with KerasClassifier
keras_model = KerasClassifier(model=create_model, epochs=10, batch_size=32, verbose=0, units =60,  dropout=0.3)

# Cast the hyperparameters to integers
param_space = {'units': (50, 60, 100),
               'dropout': (0.1, 0.5)}


# Perform Bayesian Optimization
optimizer = BayesSearchCV(keras_model, param_space, n_iter=20, cv=3)
optimizer.fit(x_train_reshaped, y_train_categorical)




/Users/apple/Documents/campus security system/.venv/lib/python3.9/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
/Users/apple/Documents/campus security system/.venv/lib/python3.9/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


BayesSearchCV(cv=3,
              estimator=KerasClassifier(batch_size=32, dropout=0.3, epochs=10, model=<function create_model at 0x133c334c0>, units=60, verbose=0),
              n_iter=20,
              search_spaces={'dropout': (0.1, 0.5), 'units': (50, 60, 100)})

In [59]:
best_hyperparameters = optimizer.best_params_

# Access the best model
best_model = optimizer.best_estimator_

print(best_hyperparameters)
print(best_model)

OrderedDict([('dropout', 0.4006260356985686), ('units', 100)])
KerasClassifier(
	model=<function create_model at 0x133aafaf0>
	build_fn=None
	warm_start=False
	random_state=None
	optimizer=rmsprop
	loss=None
	metrics=None
	batch_size=32
	validation_batch_size=None
	verbose=0
	callbacks=None
	validation_split=0.0
	shuffle=True
	run_eagerly=False
	epochs=10
	units=100
	dropout=0.4006260356985686
	class_weight=None
)


In [61]:
predictions = hyper_tuned_bilstm.predict(x_test_reshaped)

# If predictions are class probabilities, convert them to class labels
predicted_labels = np.argmax(predictions, axis=1)

# Assuming y_test_categorical contains the true labels
accuracy = accuracy_score(np.argmax(y_test_categorical, axis=1), predicted_labels)

print(f'Test Accuracy: {accuracy * 100:.4f}')

Test Accuracy: 77.2727


### Using GAUSAIN PROCESS REGRESSION FOR HYPERPARAMETER TUNING

In [66]:
param_space = {'var_smoothing': (1e-9, 1e-5)}


nb_classifier = GaussianNB()

optimizer = BayesSearchCV(nb_classifier, param_space, n_iter=20, cv=3)
optimizer.fit(x_train, y_train)

best_hyperparameters = optimizer.best_params_
print("Best Hyperparameters:", best_hyperparameters)



/Users/apple/Documents/campus security system/.venv/lib/python3.9/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
/Users/apple/Documents/campus security system/.venv/lib/python3.9/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
/Users/apple/Documents/campus security system/.venv/lib/python3.9/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
/Users/apple/Documents/campus security system/.venv/lib/python3.9/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


Best Hyperparameters: OrderedDict([('var_smoothing', 1.6124728751727817e-09)])


In [67]:
accuracy = optimizer.score(x_test, y_test)
print(f'Test Accuracy: {accuracy * 100:.4f}')

Test Accuracy: 85.9091
